In [2]:
import numpy as np

In [5]:
arr = [np.array([2, 4]), np.array([9, 0, 12, 4])]
np.unique(np.concatenate(arr))

array([ 0,  2,  4,  9, 12])

In [9]:
np.roll(np.array([0, 1, 0]), -1)

array([1, 0, 0])